## TODO:
1. Add RGB channels for $S_0$

In [50]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from dtcwt_classifier import DtcwtClassifier
from keras.datasets import mnist
import numpy

(X_train, y_train), (X_test, y_test) = mnist.load_data()

cls = DtcwtClassifier(m = 2)
cls.fit(X_train[0:300], y_train[0:300])

In [52]:
accuracy = cls.evaluate(X_test[0:100], y_test[0:100])
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 49.00


In [53]:
labels = cls.predict(X_test[0:100])

In [54]:
from sklearn.metrics import accuracy_score

print("Accuracy: ", accuracy_score(y_test[0:100], labels))

Accuracy:  0.49


In [48]:
from sklearn.svm import SVC

(X_train, y_train), (X_test, y_test) = mnist.load_data()
model = SVC(kernel="linear", probability=True)

x_train = [x.flatten() for x in X_train[0:10000]]
model.fit(x_train, y_train[0:10000])

C:\Users\sonya\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [49]:
x_test = [x.flatten() for x in X_test[0:3000]]

labels = model.predict(x_test)
print("Accuracy: ", accuracy_score(y_test[0:3000], labels))

Accuracy:  0.11333333333333333
